# Tugas 1 : Crawling Berita

**Pengertian Crawling**


Crawling adalah proses otomatisasi yang dilakukan oleh program komputer untuk menjelajahi dan mengumpulkan data dari halaman-halaman web di internet. Proses ini sering kali dilakukan oleh bot yang dikenal sebagai web crawlers atau spiders. Web crawlers ini akan menelusuri (crawl) berbagai situs web, mengakses halaman-halaman yang ada, dan mengunduh atau mengekstraksi informasi yang dibutuhkan untuk kemudian disimpan atau diindeks dalam database.

**Fungsi Crawling**



1.   Mengindeks Halaman Web:


*   Crawling adalah langkah pertama dalam proses pengindeksan halaman web oleh mesin pencari. Web crawler akan mengunjungi halaman-halaman web, membaca kontennya, dan menyimpannya ke dalam indeks mesin pencari. Ini memungkinkan mesin pencari untuk menemukan dan menampilkan halaman-halaman tersebut dalam hasil pencarian.



2.   Pengumpulan Data:

*   Crawling memungkinkan pengumpulan data dari berbagai situs web untuk keperluan tertentu, seperti analisis bisnis, riset pasar, atau pengembangan model kecerdasan buatan. Dengan crawling, data dari berbagai sumber bisa dikumpulkan secara otomatis tanpa harus melakukannya secara manual.



3. Pemantauan Perubahan Konten:

*   Dengan crawling, perubahan atau pembaruan pada suatu situs web dapat dipantau secara berkala. Ini berguna untuk aplikasi yang memerlukan informasi terbaru, seperti agregator berita, alat pemantau harga, atau layanan notifikasi.



4. SEO (Search Engine Optimization):

*   Bagi pengelola situs web, memahami bagaimana proses crawling bekerja dapat membantu dalam optimasi mesin pencari (SEO). Dengan memastikan bahwa halaman-halaman web mereka mudah di-crawl dan diindeks, mereka bisa meningkatkan kemungkinan situs web mereka muncul di hasil pencarian mesin pencari.


In [15]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Fungsi untuk mengambil data dari halaman web Detik.com
def get_data(url, kategori):
    try:
        response = requests.get(url)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return

    soup = BeautifulSoup(response.content, "html.parser")
    articles = soup.find_all("article", class_="list-content__item")

    for article in articles:
        if len(judul) >= 10:  # Hentikan jika sudah 10 berita
            return

        try:
            link = article.find("a")["href"]
            article_response = requests.get(link)
            article_response.raise_for_status()
        except (requests.exceptions.RequestException, TypeError) as e:
            print(f"Request for article failed: {e}")
            continue

        article_soup = BeautifulSoup(article_response.content, "html.parser")
        title_element = article_soup.find("h1", class_="detail__title")
        title = title_element.text.strip() if title_element else "Title Not Found"
        date_element = article_soup.find("div", class_="detail__date")
        date = date_element.text.strip() if date_element else "Date Not Found"
        content_element = article_soup.find("div", class_="detail__body-text")
        content = content_element.text.strip() if content_element else "Content Not Found"

        judul.append(title)
        tanggal.append(date)
        isi.append(content)

        print(title)
        time.sleep(1)  # Menambahkan jeda waktu 1 detik antara permintaan artikel

# Membuat list url dan kategori yang akan di-crawl
base_urls = ["https://travel.detik.com/travel-news/indeks"]
categories = ["Pariwisata"]

# Inisialisasi list untuk menyimpan data
judul = []
tanggal = []
isi = []

# Melakukan iterasi untuk setiap url dan kategori
for base_url, category in zip(base_urls, categories):
    for page in range(1, 4):  # Looping untuk beralih halaman
        if len(judul) >= 10:  # Hentikan jika sudah 10 berita
            break

        url = f"{base_url}/{page}"
        get_data(url, category)
        time.sleep(2)  # Menambahkan jeda waktu 2 detik antara permintaan halaman

# Membuat dataframe dari list data
df = pd.DataFrame({
    "judul": judul,
    "tanggal": tanggal,
    "isi": isi
})

# Menyimpan dataframe ke file CSV
df.to_csv("Crawl-berita.csv", index=False)

Rute ke Dieng Naik Bus, Terjangkau dan Lama Perjalanan
Super Air Jet Buka 3 Rute Baru dari Jogja, ke Padang-Jambi-Pangkalpinang
MotoGP Mandalika Susah Jual Tiket, Terimbas Harga Hotel Menggila
MotoGP Mandalika Sulit Bayar Hosting Fee, NTB Prioritas PON dan Pilkada
ITDC: Lowongan Volunteer MotoGP Mandalika Itu Gratis
MotoGP Mandalika Dibayangi Cacar Monyet dan Sewa Hotel Mahal
Ditinggal oleh Kelompoknya, Pendaki Tersesat, Untung Selamat
Turis Australia Tewas Setelah Diving di Manta Point
Jepang Perkenalkan Sistem Baru untuk Cegah Imigran Gelap
Jakarta di Angan-angan Ridwan Kamil: Kota Global dan Wisata Dunia


In [16]:
df=pd.read_csv("Crawl-berita.csv")
df

,judul,tanggal,isi
0,"Rute ke Dieng Naik Bus, Terjangkau dan Lama Pe...","Senin, 02 Sep 2024 14:05 WIB","Jakarta - Dieng, Wonosobo, menjadi salah satu ..."
1,"Super Air Jet Buka 3 Rute Baru dari Jogja, ke ...","Senin, 02 Sep 2024 13:53 WIB",Jakarta - Super Air Jet semakin luas rute pene...
2,"MotoGP Mandalika Susah Jual Tiket, Terimbas Ha...","Senin, 02 Sep 2024 13:05 WIB",Jakarta - Indonesia Tourism Development Corpor...
3,"MotoGP Mandalika Sulit Bayar Hosting Fee, NTB ...","Senin, 02 Sep 2024 12:05 WIB",Jakarta - InJourney Tourism Development Corpor...
4,ITDC: Lowongan Volunteer MotoGP Mandalika Itu ...,"Senin, 02 Sep 2024 11:31 WIB",Jakarta - Kabar tak sedap membayangi gelaran M...
5,MotoGP Mandalika Dibayangi Cacar Monyet dan Se...,"Senin, 02 Sep 2024 11:05 WIB",Jakarta - Penyebaran penyakit cacar monyet ata...
6,"Ditinggal oleh Kelompoknya, Pendaki Tersesat, ...","Senin, 02 Sep 2024 10:05 WIB",Jakarta - Seorang pendaki tersesat setelah dit...
7,Turis Australia Tewas Setelah Diving di Manta ...,"Senin, 02 Sep 2024 09:18 WIB","Jakarta - Seorang wisatawan asal Australia, Pa..."
8,Jepang Perkenalkan Sistem Baru untuk Cegah Imi...,"Senin, 02 Sep 2024 08:39 WIB","Jakarta - Jepang perkenalkan sistem JESTA, mew..."
9,Jakarta di Angan-angan Ridwan Kamil: Kota Glob...,"Senin, 02 Sep 2024 08:05 WIB",Jakarta - Bakal calon Gubernur DKI Jakarta 202...
